In [ ]:
import sys
sys.version

In [ ]:
import tqdm
import numpy as np
import skimage.measure
import matplotlib.pyplot as plt
%matplotlib widget

In [ ]:
import pydicom
pydicom.__version__

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import rai
rai.__version__

In [ ]:
import raicontours

from raicontours import TG263

raicontours.__version__

In [ ]:
cfg = raicontours.get_config()

In [ ]:
cfg

In [ ]:
image_paths, structure_path = rai.download_deepmind_example()
structure_path

In [ ]:
grids, image_stack, image_uids = rai.paths_to_image_stack_hfs(cfg=cfg, paths=image_paths)

In [ ]:
z_grid, y_grid, x_grid = grids

In [ ]:
models = rai.load_models(cfg=cfg)

In [ ]:
structure_ds = pydicom.read_file(structure_path)
[item.ROIName for item in structure_ds.StructureSetROISequence]

In [ ]:
align_map = {
    "Brain": [TG263.Brain],
    "Brainstem": [TG263.Brainstem],
    "Cochlea-Lt": [TG263.Cochlea_L],
    "Cochlea-Rt": [TG263.Cochlea_R],
    "Lacrimal-Lt": [TG263.Glnd_Lacrimal_L],
    "Lacrimal-Rt": [TG263.Glnd_Lacrimal_R],
    "Lens-Lt": [TG263.Lens_L],
    "Lens-Rt": [TG263.Lens_R],
    "Lung-Lt": [TG263.Lung_L],
    "Lung-Rt": [TG263.Lung_R],
    "Mandible": [TG263.Bone_Mandible],
    "Optic-Nerve-Lt": [TG263.OpticNrv_L],
    "Optic-Nerve-Rt": [TG263.OpticNrv_R],
    "Orbit-Lt": [TG263.Eye_L],
    "Orbit-Rt": [TG263.Eye_R],
    "Parotid-Lt": [TG263.Parotid_L],
    "Parotid-Rt": [TG263.Parotid_R],
    "Spinal-Cord": [TG263.SpinalCord],
    "Submandibular-Lt": [TG263.Glnd_Submand_L],
    "Submandibular-Rt": [TG263.Glnd_Submand_R],
}
structure_names = list(align_map.keys())

dicom_contours_by_structure = rai.dicom_to_contours_by_structure(
    ds=structure_ds, image_uids=image_uids, structure_names=structure_names
)

In [ ]:
step_sizes = [64, 60, 55, 50, 45, 40, 37, 33, 30, 25]

In [ ]:
collected_dice = {}    
    
for step_size in step_sizes:
    predicted_masks = rai.inference(
        cfg=cfg,
        models=models,
        image_stack=image_stack,
        max_batch_size=10,
        step_size=step_size,
    )

    predicted_contours_by_structure = rai.masks_to_contours_by_structure(
        x_grid=x_grid, y_grid=y_grid, masks=predicted_masks, structure_names=cfg["structures"]
    )

    aligned_predicted_contours_by_structure = rai.merge_contours_by_structure(
        predicted_contours_by_structure, align_map
    )

    dice = {}
    for name in align_map:
        dice[name] = rai.dice_from_contours_by_slice(
            dicom_contours_by_structure[name],
            aligned_predicted_contours_by_structure[name],
        )

    collected_dice[step_size] = dice
    print(step_size)
    print(dice)

In [ ]:
# Step-size: 50

# {'Brain': 0.9770559873907146,
#  'Brainstem': 0.7446594757682403,
#  'Cochlea-Lt': 0.0,
#  'Cochlea-Rt': 0.0,
#  'Lacrimal-Lt': 0.4287916964134596,
#  'Lacrimal-Rt': 0.7150097289790804,
#  'Lens-Lt': 0.6240236186258522,
#  'Lens-Rt': 0.6694594192976419,
#  'Lung-Lt': 0.987162664054495,
#  'Lung-Rt': 0.9883733539708488,
#  'Mandible': 0.8933116995749084,
#  'Optic-Nerve-Lt': 0.5987591575284228,
#  'Optic-Nerve-Rt': 0.7122899293396878,
#  'Orbit-Lt': 0.9375315112299282,
#  'Orbit-Rt': 0.9348481957064191,
#  'Parotid-Lt': 0.7269063671274066,
#  'Parotid-Rt': 0.737441485353478,
#  'Spinal-Cord': 0.7909805731621938,
#  'Submandibular-Lt': 0.7509269778519194,
#  'Submandibular-Rt': 0.5433672367481709}

In [ ]:
# 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.80s/it]
# 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.22s/it]
# 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:18<00:00,  4.68s/it]
# 64
# {'Brain': 0.9629588029919449, 'Brainstem': 0.6961071769445724, 'Cochlea-Lt': 0.26172024510735, 'Cochlea-Rt': 0.0, 'Lacrimal-Lt': 0.5199588145504321, 'Lacrimal-Rt': 0.6682987860661391, 'Lens-Lt': 0.49491961711729754, 'Lens-Rt': 0.49365681035142867, 'Lung-Lt': 0.9790255335979224, 'Lung-Rt': 0.9780036156764463, 'Mandible': 0.864762434291557, 'Optic-Nerve-Lt': 0.5983499856515683, 'Optic-Nerve-Rt': 0.660343128619287, 'Orbit-Lt': 0.8056716658888536, 'Orbit-Rt': 0.8093641729725283, 'Parotid-Lt': 0.7051986746607908, 'Parotid-Rt': 0.7104297404200148, 'Spinal-Cord': 0.7897631832872768, 'Submandibular-Lt': 0.7526247882007363, 'Submandibular-Rt': 0.6157494563101054}
# 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.89s/it]
# 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:07<00:00,  3.62s/it]
# 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:19<00:00,  4.91s/it]
# 60
# {'Brain': 0.9774652436126708, 'Brainstem': 0.7474377353923395, 'Cochlea-Lt': 0.1767968641915055, 'Cochlea-Rt': 0.0, 'Lacrimal-Lt': 0.23951486627412275, 'Lacrimal-Rt': 0.7158697525227612, 'Lens-Lt': 0.5514292959884519, 'Lens-Rt': 0.6368529308533324, 'Lung-Lt': 0.9759305300449723, 'Lung-Rt': 0.9885228072277406, 'Mandible': 0.8955565516480088, 'Optic-Nerve-Lt': 0.6411902563527835, 'Optic-Nerve-Rt': 0.678827657246566, 'Orbit-Lt': 0.9399953750280025, 'Orbit-Rt': 0.927951751611561, 'Parotid-Lt': 0.7442688234888652, 'Parotid-Rt': 0.758540718167435, 'Spinal-Cord': 0.8004485887468922, 'Submandibular-Lt': 0.7668204016899542, 'Submandibular-Rt': 0.6403165153754294}
# 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.85s/it]
# 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:10<00:00,  5.06s/it]
# 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:27<00:00,  3.88s/it]
# 55
# {'Brain': 0.9783380393611478, 'Brainstem': 0.7357709947884478, 'Cochlea-Lt': 0.32841816776699456, 'Cochlea-Rt': 0.0, 'Lacrimal-Lt': 0.29736451578553263, 'Lacrimal-Rt': 0.6822348337562768, 'Lens-Lt': 0.6075044170049073, 'Lens-Rt': 0.6263390124991681, 'Lung-Lt': 0.9834168910468509, 'Lung-Rt': 0.9887960559315655, 'Mandible': 0.8951125198882349, 'Optic-Nerve-Lt': 0.5847187667644942, 'Optic-Nerve-Rt': 0.6781108846568977, 'Orbit-Lt': 0.9350375553257919, 'Orbit-Rt': 0.9352668106187123, 'Parotid-Lt': 0.7498737011999762, 'Parotid-Rt': 0.7328466165218621, 'Spinal-Cord': 0.7930896905004513, 'Submandibular-Lt': 0.7814091070977186, 'Submandibular-Rt': 0.5912728727827463}
# 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.88s/it]
# 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:16<00:00,  5.57s/it]
# 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:39<00:00,  4.89s/it]
# 50
# {'Brain': 0.9766173294950212, 'Brainstem': 0.7479318749901638, 'Cochlea-Lt': 0.17410798652096243, 'Cochlea-Rt': 0.0, 'Lacrimal-Lt': 0.4142658422049932, 'Lacrimal-Rt': 0.7051573534508256, 'Lens-Lt': 0.6376929874703285, 'Lens-Rt': 0.7123772336824935, 'Lung-Lt': 0.9864649096114113, 'Lung-Rt': 0.9886692727800134, 'Mandible': 0.8946611407850689, 'Optic-Nerve-Lt': 0.6101161489648605, 'Optic-Nerve-Rt': 0.6907086212675648, 'Orbit-Lt': 0.9322248706285692, 'Orbit-Rt': 0.9353945262292207, 'Parotid-Lt': 0.7393129844626185, 'Parotid-Rt': 0.732857529468094, 'Spinal-Cord': 0.7920446878014686, 'Submandibular-Lt': 0.7586090088298545, 'Submandibular-Rt': 0.531822556463669}
# 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.75s/it]
# 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:16<00:00,  5.43s/it]
# 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:55<00:00,  6.19s/it]
# 45
# {'Brain': 0.9778362708390842, 'Brainstem': 0.7289390125487856, 'Cochlea-Lt': 0.2835384999235171, 'Cochlea-Rt': 0.0, 'Lacrimal-Lt': 0.4072143993870854, 'Lacrimal-Rt': 0.6894852861175276, 'Lens-Lt': 0.6139325176774312, 'Lens-Rt': 0.6174684451682593, 'Lung-Lt': 0.9883560854605807, 'Lung-Rt': 0.9885375291684702, 'Mandible': 0.8923329870107739, 'Optic-Nerve-Lt': 0.6086179425527166, 'Optic-Nerve-Rt': 0.68558014270524, 'Orbit-Lt': 0.9335098768262181, 'Orbit-Rt': 0.9290148908742829, 'Parotid-Lt': 0.7422630668835799, 'Parotid-Rt': 0.7342698149109725, 'Spinal-Cord': 0.7950229210780597, 'Submandibular-Lt': 0.7593562684688507, 'Submandibular-Rt': 0.47054986350334477}
# 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:14<00:00, 14.16s/it]
# 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:27<00:00,  6.97s/it]
# 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [01:13<00:00,  6.09s/it]
# 40
# {'Brain': 0.9774353995205287, 'Brainstem': 0.7518880428121287, 'Cochlea-Lt': 0.31285487566591674, 'Cochlea-Rt': 0.0, 'Lacrimal-Lt': 0.527610416628978, 'Lacrimal-Rt': 0.7123199956344667, 'Lens-Lt': 0.622374746215206, 'Lens-Rt': 0.6375602485993082, 'Lung-Lt': 0.9873273927814128, 'Lung-Rt': 0.9884155552725844, 'Mandible': 0.8963450915911084, 'Optic-Nerve-Lt': 0.5845966786685697, 'Optic-Nerve-Rt': 0.670579446635683, 'Orbit-Lt': 0.9337288614706258, 'Orbit-Rt': 0.934932256266513, 'Parotid-Lt': 0.758086099625746, 'Parotid-Rt': 0.7390492955494443, 'Spinal-Cord': 0.7873938916577, 'Submandibular-Lt': 0.7776016242489469, 'Submandibular-Rt': 0.6123807809461289}
# 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:13<00:00, 13.48s/it]
# 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:29<00:00,  7.29s/it]
# 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [01:30<00:00,  6.43s/it]
# 37
# {'Brain': 0.9767996315895476, 'Brainstem': 0.7401090755341655, 'Cochlea-Lt': 0.22856082632068736, 'Cochlea-Rt': 0.0, 'Lacrimal-Lt': 0.4412053980495106, 'Lacrimal-Rt': 0.7203508872180994, 'Lens-Lt': 0.6332220224175756, 'Lens-Rt': 0.6571194902366697, 'Lung-Lt': 0.9874728081858813, 'Lung-Rt': 0.9891623274144774, 'Mandible': 0.8980851349361075, 'Optic-Nerve-Lt': 0.6107924023399189, 'Optic-Nerve-Rt': 0.6918460753464581, 'Orbit-Lt': 0.9365144863275375, 'Orbit-Rt': 0.9345592895294098, 'Parotid-Lt': 0.7568957056108354, 'Parotid-Rt': 0.7381588848320364, 'Spinal-Cord': 0.7997901063682613, 'Submandibular-Lt': 0.7731461979684963, 'Submandibular-Rt': 0.6272837018945265}
# 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:10<00:00, 10.78s/it]
# 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:40<00:00,  6.75s/it]
# 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [02:36<00:00,  7.43s/it]
# 33
# {'Brain': 0.9784592537663463, 'Brainstem': 0.7413097726760441, 'Cochlea-Lt': 0.29083046094144843, 'Cochlea-Rt': 0.0, 'Lacrimal-Lt': 0.377907737750024, 'Lacrimal-Rt': 0.7188750675992482, 'Lens-Lt': 0.6001150501132501, 'Lens-Rt': 0.5838217685775695, 'Lung-Lt': 0.9883768984073923, 'Lung-Rt': 0.9897275616951298, 'Mandible': 0.8963714541418232, 'Optic-Nerve-Lt': 0.6269335077423134, 'Optic-Nerve-Rt': 0.6985641464520764, 'Orbit-Lt': 0.9344321760580718, 'Orbit-Rt': 0.9287775424920438, 'Parotid-Lt': 0.7367823566762346, 'Parotid-Rt': 0.7365318696809463, 'Spinal-Cord': 0.7954489547963075, 'Submandibular-Lt': 0.7623716662415342, 'Submandibular-Rt': 0.590206578143703}
# 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:19<00:00,  9.68s/it]
# 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [01:00<00:00,  8.62s/it]
# 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26/26 [04:35<00:00, 10.59s/it]
# 30
# {'Brain': 0.9786508332239935, 'Brainstem': 0.7447225141126097, 'Cochlea-Lt': 0.3160294511063905, 'Cochlea-Rt': 0.0, 'Lacrimal-Lt': 0.41227834038709155, 'Lacrimal-Rt': 0.7176156849135653, 'Lens-Lt': 0.6111765841316917, 'Lens-Rt': 0.6554802028055635, 'Lung-Lt': 0.9830171840187443, 'Lung-Rt': 0.9888323416191555, 'Mandible': 0.8985637747383387, 'Optic-Nerve-Lt': 0.6284380139348956, 'Optic-Nerve-Rt': 0.6930676376205276, 'Orbit-Lt': 0.9411754955687601, 'Orbit-Rt': 0.929505565063373, 'Parotid-Lt': 0.7360228789591557, 'Parotid-Rt': 0.7401499795438967, 'Spinal-Cord': 0.7956150411824838, 'Submandibular-Lt': 0.7695008136451815, 'Submandibular-Rt': 0.5911184814312006}

In [ ]:
# grids = (z_grid, y_grid, x_grid)
# slice_indices, axis_limits, centre_indices = rai.view_ranges_from_masks(grids, predicted_masks)

In [ ]:
# contours_by_orientation = rai.masks_to_contours_by_orientation(
#     grids, predicted_masks, structure_names=cfg["structures"]
# )

In [ ]:
# rai.draw_contours_by_orientation(
#     cfg=cfg,
#     grids=grids,
#     images=image_stack,
#     contours_by_orientation=contours_by_orientation,
#     vmin=0.2,
#     vmax=0.4,
#     axis_limits=axis_limits,
#     centre_indices=centre_indices,
# )

In [ ]:
plt.close('all')

# rai.auto_scroll_contours_by_orientation(
#     grids=grids,
#     images=image_stack,
#     contours_by_orientation=contours_by_orientation,
#     slice_indicies=slice_indices,
#     axis_limits=axis_limits,
#     structure_names=cfg["structures"],
#     vmin=0.2,
#     vmax=0.4,
# )

In [ ]:
# TODO:
# * Change figures to clickable interactive transverse/coronal/sagital bokeh
# * Calculate and report hausdorff and surface dice as well